In [1]:
# To examine the geographical distribution of job openings
# Import Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
from us import states

# Google developer API key
from config import gkey


In [2]:
# read in the csv file 
DSjob_Feb2019 = pd.read_csv('./DataSources/Data_10k_DS_Postings_USA/data_scientist_united_states_job_postings_jobspikr.csv')
DSjob_Feb2019.head()

,crawl_timestamp,url,job_title,category,company_name,city,state,country,inferred_city,inferred_state,...,job_description,job_type,salary_offered,job_board,geo,cursor,contact_email,contact_phone_number,uniq_id,html_job_description
0,2019-02-06 05:26:22 +0000,https://www.indeed.com/viewjob?jk=fd83355c2b23...,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Usa,Woodland hills,California,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819114777,NaN,NaN,3b6c6acfcba6135a31c83bd7ea493b18,NaN
1,2019-02-06 05:33:41 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Usa,Middletown,New jersey,...,We have an immediate opening for a Sharp Data ...,Undefined,NaN,dice,usa,1549432819122106,NaN,NaN,741727428839ae7ada852eebef29b0fe,NaN
2,2019-02-06 05:33:35 +0000,https://www.dice.com/jobs/detail/Data-Scientis...,Data Scientist,NaN,Cincinnati Bell Technology Solutions,New York,NY,Usa,New york,New york,...,Candidates should have the following backgroun...,Full Time,NaN,dice,usa,1549432819236156,NaN,NaN,cdc9ef9a1de327ccdc19cc0d07dbbb37,NaN
3,2019-02-06 05:33:42 +0000,https://www.indeed.com/viewjob?jk=841edd86ead2...,"Data Scientist, Aladdin Wealth Tech, Associate...",Accounting/Finance,BlackRock,New York,NY 10055 (Midtown area),Usa,New york,New york,...,Read what people are saying about working here...,Undefined,NaN,indeed,usa,1549432819259473,NaN,NaN,1c8541cd2c2c924f9391c7d3f526f64e,NaN
4,2019-02-06 05:48:23 +0000,https://job-openings.monster.com/senior-data-s...,Senior Data Scientist,biotech,CyberCoders,Charlotte,NC,Usa,Charlotte,North carolina,...,We are seeking an extraordinary Data Scientist...,Full Time,NaN,monster,usa,1549436429015957,NaN,NaN,445652a560a5441060857853cf267470,NaN


In [3]:
# first only extract the columns of concern
DSjob_Feb2019 = DSjob_Feb2019[['job_title','category','company_name','city','state',\
                              'inferred_city','inferred_state','post_date','job_description',\
                              'job_type','job_board']]
DSjob_Feb2019.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        10000 non-null  object
 1   category         9118 non-null   object
 2   company_name     9999 non-null   object
 3   city             9751 non-null   object
 4   state            9584 non-null   object
 5   inferred_city    8980 non-null   object
 6   inferred_state   9484 non-null   object
 7   post_date        10000 non-null  object
 8   job_description  10000 non-null  object
 9   job_type         10000 non-null  object
 10  job_board        9310 non-null   object
dtypes: object(11)
memory usage: 859.5+ KB


In [4]:
DSjob_Feb2019["city_state"] = DSjob_Feb2019["inferred_city"] + ', '+ DSjob_Feb2019["inferred_state"]
DSjob_Feb2019.head(2)

,job_title,category,company_name,city,state,inferred_city,inferred_state,post_date,job_description,job_type,job_board,city_state
0,Enterprise Data Scientist I,Accounting/Finance,Farmers Insurance Group,Woodland Hills,CA,Woodland hills,California,2019-02-06,Read what people are saying about working here...,Undefined,indeed,"Woodland hills, California"
1,Data Scientist,NaN,Luxoft USA Inc,Middletown,NJ,Middletown,New jersey,2019-02-05,We have an immediate opening for a Sharp Data ...,Undefined,dice,"Middletown, New jersey"


In [5]:
# grouped_citystate = DSjob_Feb2019.groupby('city_state')['category'].count().sort_values(ascending=False)
grouped_citystate = DSjob_Feb2019.groupby('city_state')['category'].count().sort_values(ascending=False).reset_index()
grouped_citystate.head(300)


,city_state,category
0,"New york, New york",643
1,"San francisco, California",402
2,"Chicago, Illinois",261
3,"Seattle, Washington",259
4,"Boston, Massachusetts",232
...,...,...
295,"Sterling, Virginia",4
296,"Topeka, Kansas",3
297,"Berwyn, Pennsylvania",3
298,"Berwick, Maine",3


In [7]:
grouped_citystate['lat'] = 999
grouped_citystate['lng']= 999
grouped_citystate

,city_state,category,lat,lng
0,"New york, New york",643,999,999
1,"San francisco, California",402,999,999
2,"Chicago, Illinois",261,999,999
3,"Seattle, Washington",259,999,999
4,"Boston, Massachusetts",232,999,999
...,...,...,...,...
731,"Miramar, Florida",0,999,999
732,"Des plaines, Illinois",0,999,999
733,"Sandy springs, Georgia",0,999,999
734,"Santa clarita, California",0,999,999


In [9]:
for irow in np.arange(len(grouped_citystate)):
    print(irow)
    target_city = grouped_citystate.city_state[irow]
    # Build the endpoint URL
    target_url = f'https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={gkey}'
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
#     print(json.dumps(geo_data, indent=4, sort_keys=True))
    # Extract latitude and longitude
    try:
        grouped_citystate.loc[irow, 'lat'] = geo_data["results"][0]["geometry"]["location"]["lat"]
        grouped_citystate.loc[irow, 'lng'] = geo_data["results"][0]["geometry"]["location"]["lng"]
    except:
        grouped_citystate.loc[irow, 'lat'] = '999'
        grouped_citystate.loc[irow, 'lng'] = '999'

        

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
grouped_citystate

,city_state,category,lat,lng
0,"New york, New york",643,40.712775,-74.005973
1,"San francisco, California",402,37.774929,-122.419415
2,"Chicago, Illinois",261,41.878114,-87.629798
3,"Seattle, Washington",259,47.606209,-122.332071
4,"Boston, Massachusetts",232,42.360082,-71.058880
...,...,...,...,...
731,"Miramar, Florida",0,25.986076,-80.303560
732,"Des plaines, Illinois",0,42.033362,-87.883399
733,"Sandy springs, Georgia",0,33.930435,-84.373315
734,"Santa clarita, California",0,34.391664,-118.542586


In [29]:
# Create a job opening Heatmap layer
locations = grouped_citystate[["lat", "lng"]].astype(float)
locations

,lat,lng
0,40.712775,-74.005973
1,37.774929,-122.419415
2,41.878114,-87.629798
3,47.606209,-122.332071
4,42.360082,-71.058880
...,...,...
731,25.986076,-80.303560
732,42.033362,-87.883399
733,33.930435,-84.373315
734,34.391664,-118.542586


In [20]:
grouped_citystate.category

0      643
1      402
2      261
3      259
4      232
      ... 
731      0
732      0
733      0
734      0
735      0
Name: category, Length: 736, dtype: int64

In [34]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=grouped_citystate.category, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig
# fig.savefig("./Images/Heatmap_JobOpening_Geographic_Distribution.png", bbox_inches='tight')

Figure(layout=FigureLayout(height='420px'))